# Especializar um Modelo Pré-treinado do Hugging Face

Este notebook apresenta um roteiro que detalha os passos necessários para utilizar e especializar modelos de linguagem pré-treinados para a tarefa de sumarização de textos em português, garantindo a instalação e configuração adequadas das bibliotecas necessárias, bem como a seleção e utilização de modelos apropriados para o idioma.

### 1. Instalação de dependências
Execute o seguinte bloco para instalar e atualizar as dependências. Em seguida reinicie o kernel antes de executar os demais blocos.

In [1]:
%%capture
!pip install transformers datasets torch
!pip install sentencepiece
!pip install tf-keras
!pip install transformers[torch] -U
!pip install accelerate -U
!pip install transformers datasets torch accelerate -U

### 2. Carregamento do Corpus e Pré-processamento

In [2]:
from datasets import load_dataset, Dataset

# Carrega o corpus do arquivo gerado pelo outro notebook deste repositório.
dataset = load_dataset('csv', data_files='combined_corpus.csv')

In [3]:
from datasets import DatasetDict

# Dividi o dataset em 90% para treino e 10% para validação
train_test_split = dataset['train'].train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test']
})

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'texto', 'summary'],
        num_rows: 82
    })
    validation: Dataset({
        features: ['id', 'texto', 'summary'],
        num_rows: 10
    })
})


### 3. Escolha do Modelo Pré-Treinado

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "rhaymison/flan-t5-portuguese-small-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

### 4. Tokenização e Preparação dos Dados

In [5]:
def preprocess_function(examples):
    inputs = [doc for doc in examples['texto']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=128, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/82 [00:00<?, ? examples/s]

c:\Users\jcf_s\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

### 5. Configuração do Treinamento

In [6]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer
)

c:\Users\jcf_s\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### 6. Treinamento do Modelo

In [7]:
trainer.train()

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.0828173160552979, 'eval_runtime': 6.1723, 'eval_samples_per_second': 1.62, 'eval_steps_per_second': 0.324, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.0470554828643799, 'eval_runtime': 6.1401, 'eval_samples_per_second': 1.629, 'eval_steps_per_second': 0.326, 'epoch': 2.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.032470941543579, 'eval_runtime': 6.3361, 'eval_samples_per_second': 1.578, 'eval_steps_per_second': 0.316, 'epoch': 3.0}
{'train_runtime': 738.1138, 'train_samples_per_second': 0.333, 'train_steps_per_second': 0.045, 'train_loss': 1.342020323782256, 'epoch': 3.0}


TrainOutput(global_step=33, training_loss=1.342020323782256, metrics={'train_runtime': 738.1138, 'train_samples_per_second': 0.333, 'train_steps_per_second': 0.045, 'total_flos': 45729064157184.0, 'train_loss': 1.342020323782256, 'epoch': 3.0})

### 7. Avaliação e Ajustes Finais

In [8]:
from pprint import pprint

results = trainer.evaluate()

pprint(results, indent=2)

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.032470941543579, 'eval_runtime': 6.294, 'eval_samples_per_second': 1.589, 'eval_steps_per_second': 0.318, 'epoch': 3.0}


### 8. Registrando o Modelo

In [9]:
model.save_pretrained('./final_model')
tokenizer.save_pretrained('./final_model')

('./final_model\\tokenizer_config.json',
 './final_model\\special_tokens_map.json',
 './final_model\\tokenizer.json')

### 9. Uso do Modelo

In [15]:
from transformers import pipeline

# Carregar o modelo treinado
summarizer = pipeline('summarization', model='./final_model', tokenizer='./final_model')

# Caminho para o arquivo de teste
test_file_path = 'input-test.txt'

# Ler todo o conteúdo do arquivo de teste
with open(test_file_path, 'r', encoding='utf-8') as file:
    test_text = file.read().strip()

# Sumarizar o texto do arquivo
summary = summarizer(test_text)[0]['summary_text']

# Exibir o resumo
print("Resumo do texto:")
print(summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (5653 > 512). Running this sequence through the model will result in indexing errors


Resumo do texto:
Setor de Defesa é aprovado pela Resoluço ConSUG/MD no 6, de 11 de dezembro de 2020, e especialistas apontam que o Plano Estratégico Setorial 2024-2027 está alinhado com a cadeia de valor setorial, adequadamente preparadas e fortalecimento dos objetivos nacionais, como a primeira esforço da cidadania. Resultado de estudos e debates sobre temas ligados ao enfrentamento nacional e defeso no significa exclusividade para o desenvolvimento


Setor de Defesa é aprovado pela Resoluço ConSUG/MD no 6, de 11 de dezembro de 2020, e especialistas apontam que o Plano Estratégico Setorial 2024-2027 está alinhado com a cadeia de valor setorial, adequadamente preparadas e fortalecimento dos objetivos nacionais, como a primeira esforço da cidadania. Resultado de estudos e debates sobre temas ligados ao enfrentamento nacional e defeso no significa exclusividade para o desenvolvimento